In [252]:
import SimFunctions
import SimRNG 
import SimClasses
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
import scipy.stats as stats

In [253]:
import xlrd
# Read the data file we got through data cleaning. 
wb = xlrd.open_workbook('BikeRentCount.xlsx')
# The file consists of five sheets and each sheet contains data for one station.
sheets = wb.sheet_names()
MaxRentingRate=[]
RentingRate=[]
for i in range(len(sheets)):
    data = pd.read_excel('BikeRentCount.xlsx', sheet_name=sheets[i])
    #Calculate the average number of rented bikes for each twenty minutes of each station.
    rentingRates = data.mean()
    RentingRate.append(rentingRates)
    # Find the maximum of these rates that would be used for function NSPP
    maxRentingRate = max(rentingRates)
    MaxRentingRate.append(maxRentingRate)
    
# Example: the number of rented bikes at Station 2 for every 20 minutes (indexing from 0) 
RentingRate[1]

7:00-7:20     1.869565
7:20-7:40     4.967391
7:40-8:00     7.923913
8:00-8:20     4.619565
8:20-8:40     6.086957
8:40-9:00     4.717391
9:00-9:20     3.478261
9:20-9:40     3.217391
9:40-10:00    1.706522
dtype: float64

In [254]:
# Example: the number of rented bikes at Station 2(indexing from 0) during time 8:40-9:00
RentingRate[1][5]

4.717391304347826

In [255]:
# Example: the maximum number of rented bikes at station 2(indexing from 0)
MaxRentingRate[1]

7.923913043478261

In [256]:
# Similar data processing for 'BikeReturnCount.xlsx'
wb = xlrd.open_workbook('BikeReturnCount.xlsx')
sheets = wb.sheet_names()
MaxReturnRate=[]
ReturnRate=[]
for i in range(len(sheets)):
    data = pd.read_excel('BikeReturnCount.xlsx', sheet_name=sheets[i])
    returnRates = data.mean()
    ReturnRate.append(returnRates)
    maxReturnRate = max(returnRates)
    MaxReturnRate.append(maxReturnRate)

# Simulation constants

In [318]:
order_threshold = 5.0 #schedule the "rebalancing" once the number of bikes is less than the threshold
order_up_to = 10.0 # the number of bikes in the station would reach 5 after rebalancing
delivery_delay = 20 # it takes 20 minutes for bikes to be transported to the station for bike rebalancing
SIM_RUN = 10 #number of simulation runs
operation_cost = 2 # USD per bike for operation
oil_gas = 3 # USD per 1 refillment
service_fee = 3.75 # USD per bike per ride
PENALTY = 3 # CAD for cost of loss of business oportunity
loss_profit = 0.01 # CAD per bike per minute for loss of business opportunity
RunLength = 180 # 3 hours

In [371]:
def Rental(**kwargs):
    global revenue, penalty, Loss_profit
    #Check which station the rental event occurs in
    station_id = kwargs['stationID']
    Pickup_queue = kwargs["pickup_queue"]
    Dropoff_queue = kwargs["dropoff_queue"]
    #Use NSPP to schedule the next bike rental event for current station
    interarrival = NSPP(station_id,MaxRentingRate,RentingRate)
    SimFunctions.Schedule(Calendar,"Rent a bike",interarrival,stationID = station_id,pickup_queue = TheQueues[station_id],dropoff_queue = TheQueues[station_id+5])

    # Checks if there are people waiting to put the bikes back.
    if Dropoff_queue.NumQueue() > 0:
        DepartingCustomer = Dropoff_queue.Remove()
        Waiting_time = SimClasses.Clock - DepartingCustomer.CreateTime
        Wait.Record(SimClasses.Clock - DepartingCustomer.CreateTime)
        # If customer has waited too long to return the bike, refund is given.
        # The customer gets the bike from the people waiting to put the bikes back.
        if Waiting_time >5:
            penalty += service_fee
            
    #If no one is waiting to put the bikes back,
    #check if there are bikes available and update number of bikes for the station.
    else:
        if Num_bikes[station_id]>0:
            Num_bikes[station_id]-=1
            # Customer pays to rent bike.
            revenue += service_fee
        #No bikes available so the customer begins waiting for bikes to become available.
        else:
            Customer = SimClasses.Entity()
            Pickup_queue.Add(Customer)
            
    #Schedule the bike rebalancing event once the number of bikes is less than the threshold 
    """
    There are two conditions to be met for the rebalancing operation. Schedule the rebalancing operation 
    once the number of bikes is less than the threshold and the previous rebalance operation has ended. 
    """
    if Num_bikes[station_id] <= order_threshold and quantity[station_id] == 0:
        quantity[station_id] = order_up_to - Num_bikes[station_id]
        SimFunctions.Schedule(Calendar,"rebalancing",delivery_delay,stationID=station_id,pickup_queue = TheQueues[station_id],dropoff_queue = TheQueues[station_id+5],num_ordered=quantity[station_id], signal= -1)

In [372]:
def RideEnd(**kwargs):
    global  revenue, penalty, Loss_profit
    station_id = kwargs['stationID']
    Dropoff_queue = kwargs['dropoff_queue']
    Pickup_queue = kwargs['pickup_queue']
     ##Use NSPP to schedule the next end of ride for current station
    interarrival = NSPP(station_id,MaxReturnRate,ReturnRate)
    SimFunctions.Schedule(Calendar,"Return a bike",interarrival,stationID = station_id,pickup_queue = TheQueues[station_id],dropoff_queue = TheQueues[station_id+5])
    
    # We assume not every customer will wait for a bike and eventually take a bike.Customers leave after 5 mins.
    while(True):
        # Check if customers are waiting.
        if Pickup_queue.NumQueue() == 0:
            #If no one is waiting to pick up the bike,check if there are empty racks to keep the bike.   
            if Num_bikes[station_id] < Num_docks[station_id]:
                # Customer returns the bike to the rack.
                Num_bikes[station_id] += 1
            # No empty racks. The customer begins waiting in queue.
            else:
                Customer = SimClasses.Entity()
                Dropoff_queue.Add(Customer)
            break
        #If there are customers waiting to pick up the bike,
        #check if the customer has left due to waiting too long to pick up the bike.
        DepartingCustomer = Pickup_queue.Remove()
        Waiting_time = SimClasses.Clock - DepartingCustomer.CreateTime
        Wait.Record(SimClasses.Clock - DepartingCustomer.CreateTime)
        if Waiting_time < 5:
            # Next waiting customer gets a bike from the customer who has finished the ride.
            # Break the while loop.
            break
        else:
            # We lose a customer because the customer has waited too long
            penalty += PENALTY
            
    if Dropoff_queue.NumQueue() >= order_threshold and quantity[station_id] == 0:
        quantity[station_id] = Dropoff_queue.NumQueue()
        SimFunctions.Schedule(Calendar,"rebalancing",delivery_delay,stationID=station_id,pickup_queue = TheQueues[station_id],dropoff_queue = TheQueues[station_id+5],num_ordered=quantity[station_id], signal= 1)

In [378]:
def rebalancing(**kwargs):
    global revenue , cost, Loss_profit,penalty
    station_id = kwargs['stationID']
    Dropoff_queue = kwargs['dropoff_queue']
    Pickup_queue = kwargs['pickup_queue']
    #The number of bikes needed for bike rebalancing
    Num_ordered = kwargs['num_ordered']
    Signal = kwargs['signal']
    #Cost due to bike rebalanccing
    cost += (Num_ordered * operation_cost) + oil_gas 
    # Signal is -1 means the station is almost out of bikes for customers renting bikes, 
    # then the center will carry bikes to the station 
    if Signal == -1:
        #The number of bikes changes after the rebalancing operation
        Num_bikes[station_id] += Num_ordered
        # We assume not every customer will wait for a bike and eventually take a bike..Customers leave after 5 mins.
        while(True):
            # Check if customers are waiting in the pick_up queue and if the ordered number of bikes is in short supply.
            if (Pickup_queue.NumQueue() == 0.0) | (Num_bikes[station_id] == 0.0):
                break

            DepartingCustomer = Pickup_queue.Remove()
            Waiting_time = SimClasses.Clock - DepartingCustomer.CreateTime
            Wait.Record(SimClasses.Clock - DepartingCustomer.CreateTime)
            if Waiting_time < 5:
                # Next waiting customer gets a bike
                 Num_bikes[station_id] -= 1
            else:
                # We lose a customer
                penalty += PENALTY
    # Signal is +1 means the station is out of racks for customers returning bikes, 
    # then the center will carry bikes back to the center             
    else:
        while(True):
            # Check if customers are waiting in the Dropoff queue
            if (Dropoff_queue.NumQueue() == 0.0)| (Num_ordered==0.0):
                break

            DepartingCustomer = Dropoff_queue.Remove()
            Num_ordered-=1
            Waiting_time = SimClasses.Clock - DepartingCustomer.CreateTime
            Wait.Record(SimClasses.Clock - DepartingCustomer.CreateTime)
            if Waiting_time > 5:
                # If customer has waited too long to return the bike, refund is given.
                 penalty += service_fee
        Num_bikes[station_id] -= Num_ordered
    #Set the num_oredered to zero so the current rebalancing operation is over.
    quantity[station_id] = 0

In [379]:
# Prevent index out of range

def PW_ArrRate(p,t,Rates):
    hour = int(t/20)
    if hour <= 8:
        return Rates[p][hour]
    else:
        return Rates[p][-1]


In [380]:
def NSPP(q,MaxRate,Rate):
    PossibleArrival = SimClasses.Clock + SimRNG.Expon(1/(MaxRate[q]/20), 1)
    while SimRNG.Uniform(0, 1, 1) >= PW_ArrRate(q,PossibleArrival,Rate)/(MaxRate[q]):
        PossibleArrival = PossibleArrival + SimRNG.Expon(1/(MaxRate[q]/20), 1)
    nspp = PossibleArrival - SimClasses.Clock
    return nspp

In [381]:
# Get all trial solution 
trial_solution = []
for a in range(5,15):
    for b in range(5,27):
        for c in range(5,11):
            for d in range(5,15):
                for e in range(5,15):
                    # Initial number of bikes available in the system.
                    if (a+b+c+d+e)==70:
                        trial_solution.append([a,b,c,d,e])
len(trial_solution)

480

In [382]:
Calendar = SimClasses.EventCalendar()
Wait = SimClasses.DTStat()

Pickup_Queue1 = SimClasses.FIFOQueue()
Pickup_Queue2 = SimClasses.FIFOQueue()
Pickup_Queue3 = SimClasses.FIFOQueue()
Pickup_Queue4 = SimClasses.FIFOQueue()
Pickup_Queue5 = SimClasses.FIFOQueue()

Dropoff_Queue1 = SimClasses.FIFOQueue()
Dropoff_Queue2 = SimClasses.FIFOQueue()
Dropoff_Queue3 = SimClasses.FIFOQueue()
Dropoff_Queue4 = SimClasses.FIFOQueue()
Dropoff_Queue5 = SimClasses.FIFOQueue()

TheCTStats = []
TheDTStats = []
TheQueues = []
TheResources = []

TheQueues.append(Pickup_Queue1)
TheQueues.append(Pickup_Queue2)
TheQueues.append(Pickup_Queue3)
TheQueues.append(Pickup_Queue4)
TheQueues.append(Pickup_Queue5)
TheQueues.append(Dropoff_Queue1)
TheQueues.append(Dropoff_Queue2)
TheQueues.append(Dropoff_Queue3)
TheQueues.append(Dropoff_Queue4)
TheQueues.append(Dropoff_Queue5)

TheDTStats.append(Wait)

AllWaitMean = np.zeros((SIM_RUN,len(trial_solution)))

ZSimRNG = SimRNG.InitializeRNSeed()

#Create an 2d array to record the balance of each trial solution in each simulation run
output = np.zeros((SIM_RUN,len(trial_solution)))
for k in range(SIM_RUN):
    for j in range(len(trial_solution)):
        Num_bikes=[]
        initial_Num_bikes=[]
        #Apply the our model to each trial solution
        for i in range(0,5,1):
            Num_bikes.append(trial_solution[j][i])
            initial_Num_bikes.append(trial_solution[j][i])
        # Number of bike racks (total) at each station. This is the maximum parking
        # capacity for every station.
        Num_docks=[15,27,11,15,15]
        #initial number of ordered bikes for rebalancing
        quantity=[0,0,0,0,0]
        revenue = 0
        cost = 0
        penalty = 0
        Loss_profit = 0

        SimClasses.Clock = 0.0
        SimFunctions.SimFunctionsInit(Calendar,TheQueues,TheCTStats,TheDTStats,TheResources)
        for i in range(0,5,1):
            #Use NSPP to schedule the first arrival for each station.
            #Initialize queues at each station.
            SimFunctions.Schedule(Calendar,"Rent a bike",NSPP(i,MaxRentingRate,RentingRate),stationID = i,pickup_queue = TheQueues[i],dropoff_queue = TheQueues[i+5])
            SimFunctions.Schedule(Calendar,"Return a bike",NSPP(i,MaxReturnRate,ReturnRate),stationID = i,pickup_queue = TheQueues[i],dropoff_queue = TheQueues[i+5]) 
        SimFunctions.Schedule(Calendar,"EndSimulation",RunLength)
        NextEvent = Calendar.Remove()
        SimClasses.Clock = NextEvent.EventTime
        if NextEvent.EventType == "Rent a bike":
            Rental(**NextEvent.kwargs)
        elif NextEvent.EventType == "Return a bike":
            RideEnd(**NextEvent.kwargs)
        elif NextEvent.EventType == "rebalancing":
            rebalancing(**NextEvent.kwargs)

        while NextEvent.EventType != "EndSimulation":
            NextEvent = Calendar.Remove()
            SimClasses.Clock = NextEvent.EventTime
            if NextEvent.EventType == "Rent a bike":
                Rental(**NextEvent.kwargs)
            elif NextEvent.EventType == "Return a bike":
                RideEnd(**NextEvent.kwargs)
            elif NextEvent.EventType == "rebalancing":
                rebalancing(**NextEvent.kwargs)
        repositioning_bikes= (abs(np.array(Num_bikes)-np.array(initial_Num_bikes)).sum())
        cost += (repositioning_bikes * operation_cost) + oil_gas 
        balance = revenue - cost - penalty - Loss_profit
        output[k][j]= balance
        AllWaitMean[k][j] = Wait.Mean()




In [383]:
output

array([[149.75, 122.  , 122.5 , ..., 130.5 , 184.75, 109.  ],
       [ 77.25, 159.  , 141.25, ..., 183.25, 182.25, 103.  ],
       [148.  , 117.75, 181.75, ..., 183.  , 129.5 , 201.  ],
       ...,
       [129.5 , 131.5 , 144.  , ..., 166.25,  72.75,  69.75],
       [164.75, 141.25, 112.5 , ..., 130.75, 138.75, 206.  ],
       [115.25, 164.75, 150.  , ..., 194.5 , 110.25, 157.5 ]])

In [389]:
AllWaitMean

array([[23.21249451, 28.28540802, 37.01267943, ..., 23.44500593,
         4.33035119, 29.4044925 ],
       [32.54382519, 16.34786507,  6.84090783, ...,  0.        ,
        60.25112755, 30.99510175],
       [17.55548069, 39.69494135,  0.        , ..., 11.08027038,
         9.99187396,  0.        ],
       ...,
       [ 7.1130627 ,  8.88566258, 10.70282784, ...,  9.12929153,
        17.44885468, 35.90025892],
       [ 0.19580383,  2.11226214, 14.41100774, ...,  0.        ,
         0.        ,  8.01076486],
       [38.77886165,  1.50146179, 15.1446977 , ...,  0.        ,
        19.32024466, 10.71327351]])

In [385]:
#Get the the expectation by calculating the mean over all simulation runs for each trial solution
Estimated_Expected_waittime=np.mean(AllWaitMean,axis=0)
#Get the best solution to minimize customers waiting time. 
trial_solution[np.argmin(Estimated_Expected_waittime)]

[11, 25, 10, 11, 13]

In [386]:
#Get the the expectation by calculating the mean over all simulation runs for each trial solution
Estimated_Expected_balance=np.mean(output,axis=0)
#Get the best solution to maximize the balance.
trial_solution[np.argmax(Estimated_Expected_balance)]

[14, 24, 10, 11, 11]

In [387]:
#the estimated expected balance of all trial solutions
Estimated_Expected_balance

array([139.55 , 142.775, 145.375, 144.65 , 168.25 , 147.1  , 146.7  ,
       169.25 , 167.225, 138.15 , 148.475, 146.725, 143.5  , 155.1  ,
       160.35 , 155.675, 161.775, 166.2  , 158.65 , 160.125, 143.1  ,
       161.325, 168.675, 148.6  , 159.625, 152.925, 151.525, 140.425,
       172.025, 151.05 , 141.1  , 176.225, 166.075, 157.725, 155.225,
       157.15 , 152.1  , 162.525, 162.45 , 159.075, 162.775, 165.1  ,
       168.85 , 168.05 , 172.1  , 147.325, 148.325, 159.35 , 172.775,
       154.375, 175.475, 148.6  , 176.575, 175.75 , 175.9  , 136.825,
       125.7  , 149.275, 171.5  , 159.325, 158.525, 186.825, 165.75 ,
       166.275, 144.925, 172.725, 179.625, 152.75 , 171.3  , 162.625,
       179.05 , 155.125, 167.3  , 161.1  , 166.8  , 157.45 , 167.925,
       177.775, 171.725, 167.275, 133.9  , 163.7  , 157.475, 176.425,
       173.3  , 163.425, 175.475, 177.075, 149.95 , 157.625, 155.45 ,
       148.975, 131.15 , 159.825, 173.125, 164.675, 151.75 , 160.275,
       171.725, 144.